In [1]:
import numpy as np
import pandas as pd
from scipy.stats import multivariate_normal
import scipy.sparse as sp	
from statsmodels.api import Logit

import gurobipy as gp
from gurobipy import GRB

This is an implementation of the simulation section of Yu's Balancing Weights Causal Inference in Observational Factorial Studies paper (Yu 5.1).

### Data Initialization

In [2]:
rseed = 42
np.random.seed(rseed)
n = 500 # set as 500, 1000, 2000
rho = 0 # set as 0, 0.2, 0.4 for each scenario

Following the specification in Yu: "the treatment assignment mechanism for $Z_{ik}$ is independent across $k$'s and satisfies a logistic regression that $P(Z_{ik} = 1) = \frac{1}{(1+exp(-\beta_k^T X_i))}$ where "$\beta_1 = (\frac{1}{4}, \frac{2}{4}, 0, \frac{3}{4}, 1)$, $\beta_2 = (\frac{3}{4}, \frac{1}{4}, 1, 0, \frac{2}{4})$, $\beta_3 = (1, 0, \frac{3}{4}, \frac{2}{4}, \frac{1}{4})$."

This treatment assignment ensures all $2^3=8$ treatment combination groups are non-empty and observed so that the paper's proposed weighting estimators are applicable. We additionally assume conditional independence of factors given covariates, and that only the main factors of the three treatments are non-negligible.

In [3]:
# Define mean vector (mu)
mu = np.array([0.1, 0.1, 0.1, 0, 0]).T

# Define covariance matrix (Sigma) with 5 covariates, defined as according to paper specifications
#   - Diagonal filled with ones, rest with correlation coefficient rho
Sigma = np.full((5, 5), rho)
np.fill_diagonal(Sigma, 1)  

# Generate covariates (X) from multivariate normal
np.random.seed(rseed)
X = multivariate_normal.rvs(mean=mu, cov=Sigma, size=n)

In [4]:
# Define beta coefficients for treatment assignments to fulfill all treatment outcomes observed condition
beta_1 = np.array([1/4, 2/4, 0, 3/4, 1])
beta_2 = np.array([3/4, 1/4, 1, 0, 2/4])
beta_3 = np.array([1, 0, 3/4, 2/4, 1/4])

# Logistic function to generate treatment assignments
def logistic_prob(X, beta):
    return 1 / (1 + np.exp(-X @ beta))

# Generate treatment assignments independently
Z1 = np.random.binomial(1, logistic_prob(X, beta_1), size=n)
Z2 = np.random.binomial(1, logistic_prob(X, beta_2), size=n)
Z3 = np.random.binomial(1, logistic_prob(X, beta_3), size=n)

For our problem setup, we next consider three outcome models, with errors following standard normal distribution.
    
An additive outcome: 
$Y_{i1} = 2\sum_{k=1}^5 X_{ik} + \sum_{j=1}^3 Z_{ij} + \epsilon_{i1}$ 
    
A heterogeneous treatment effect outcome: 
$Y_{i2} = 2\sum_{k=1}^5 X_{ik} + \sum_{k=1}^5 X_{ik} \sum_{j=1}^3 Z_{ij} + \epsilon_{i2}$ 
    
A misspecified outcome:
$Y_{i3} = sin(X_{i1}) + cos(X_{i2}) + (min(1, X_{i1}) + X_{i2})Z_{i1} + \sum_{k=1}^5 X_{ik} \sum_{j=2}^3 Z_{ij} + \epsilon_{i3}$

Additionally, four estimators are implemented for each main effect $\tau_k, k=1, 2, 3$ under each outcome model:
- The additive regression estimator
- The interaction regression estimator
- The weighting estimator under general additive model assumption and covariate basis functions $h_s(X) = X_s, s=1,... 5$
- The weighting estimator with balance constraints under the outcome model specification with treatment effect heterogeneity, and the same basis functions

https://par.nsf.gov/servlets/purl/10337012 - Supplementary reading to learn more about regression based methods causal inference with factorial experiments

### Outcome Model Simulation

In [5]:
np.random.seed(rseed)
# From top to bottom: defining general, heterogeneous treatment effect, and misspecified outcome models
Y1 = 2*np.sum(X, axis=1) + Z1 + Z2 + Z3 + np.random.normal(0, 1, n)
Y2 = 2*np.sum(X, axis=1) + np.sum(X, axis=1)*(Z1 + Z2 + Z3) + np.random.normal(0, 1, n)
Y3 = np.sin(X[:,0]) + np.cos(X[:,1]) + (np.minimum(1, X[:,0]) + X[:,1]) * Z1 + np.sum(X, axis=1) * (Z2 + Z3) + np.random.normal(0, 1, n)

Using OLS for implementing additive regression $\hat{\beta} = (X^T X)^-1 X^T Y$ on all three outcome models. Specifically, we yield the additive regression main effect estimates for $\tau_k, k=1, 2, 3$ by regressing our outcome variable $Y$, specified by outcome model, on the covariates $X$ and treatments $Z$ for $j=1,...,5$, $k=1, 2, 3$ and multiplying the coefficient for Z by 2.

In [6]:
# Implementing the additive regression model:
X_design = np.column_stack(((np.ones(n)), X, Z1, Z2, Z3))
for i, y in enumerate([Y1, Y2, Y3]):
    beta_hat = np.linalg.inv(X_design.T @ X_design) @ X_design.T @ y
    print(f"\nEstimated factorial effects (additive model) for Y{i+1}", 2*beta_hat[6:])


Estimated factorial effects (additive model) for Y1 [1.75483927 1.83782051 2.04853169]

Estimated factorial effects (additive model) for Y2 [1.78935311 0.25075834 1.62555032]

Estimated factorial effects (additive model) for Y3 [0.77700929 0.55546364 1.37131804]


And now adding interaction terms between covariates and treatment assignments, although we only recover the main effects as non-negligible coefficients.

In [7]:
# Implementing the interaction regression model:
XZ_interaction_terms = np.hstack([(X[:, j:j+1] * Z1.reshape(-1, 1)) for j in range(5)] +
                              [(X[:, j:j+1] * Z2.reshape(-1, 1)) for j in range(5)] +
                              [(X[:, j:j+1] * Z3.reshape(-1, 1)) for j in range(5)])

X_design = np.column_stack(((np.ones(n)), X, Z1, Z2, Z3, XZ_interaction_terms))

# Displaying estimated coefficients
for i, y in enumerate([Y1, Y2, Y3]):
    beta_hat = np.linalg.inv(X_design.T @ X_design) @ X_design.T @ y
    print(f"\nEstimated factorial effects (interaction model) for Y{i+1}", 2*beta_hat[6:9])


Estimated factorial effects (interaction model) for Y1 [1.76217942 1.85778257 2.02815107]

Estimated factorial effects (interaction model) for Y2 [ 0.36005522 -0.37283454  0.31776519]

Estimated factorial effects (interaction model) for Y3 [-0.20459922  0.06696588  0.19336125]


Now using Gurobi to implement the proposed weighting method, but first we have to set up the balancing constraints. We simulate using both the additive balance constraints and the interaction balance constraints. We use basis functions $h_s(X) = X_s$, meaning basis functions as just the unmodified, corresponding covariate column to each $s\in S $; the raw value of the covariate. From this we define the following balancing constraints:

A contrast vector $g_k=(g_{kz})_{z \in Z}$, a $2^K$ dimensional vector with half +1s and half -1s, indicating if a combination has $z_k = +1$ or $z_k = -1$. Given that we're only considering main effects we only devise a contrast vector accounting for all $2^3 = 8$ possible treatment combinations, with 3 binary factors.

Obtain $g_{Kz}$, the contrast coefficient of the expected potential outcome under treatment combination $z$, as $g_Kz = g_{Kz}^{+} - g_{Kz}^{-}$. $g_{Kz}^{+} = max(g_{Kz}, 0)$ and $g_{Kz}^{-} = max(-g_{Kz}, 0)$.


In [8]:
# Define contrasts vector as all possible treatment assignments
g = np.array([[1, 1, 1, 1, -1, -1, -1, -1],  # Each row is a unique treatment combination from 3 binary factors, each column is a factor
             [1, 1, -1, -1, 1, 1, -1, -1],
             [1, -1, 1, -1, 1, -1, 1, -1]])
g_p = np.maximum(g, 0)  # Computes element wise maximum
g_m = np.maximum(-g, 0)

Finally, let $I(Z_i = z)$ be an indicator for whether an individual $i$ received treatment combination $z$.

We then let $A_{iK}^{\Omega} = \sum_{z\in Z} g_{Kz}^{\Omega} I(Z_i = z)$ denote whether an individual belongs to the positive or negative part of the contrast $g_K$, with $\Omega=+, -$.

In [9]:
# Define indicator vector -- since we only have the single contrast vector for main effects, have two A vectors
p = g.shape[0]
A_p = np.zeros((n, p))
A_m = np.zeros((n, p))
for i in range(n): # iterate through all individuals
    # obs = (Z1[i], Z2[i], Z3[i])
    ind = -1
    for j in range(g.shape[1]):
        if Z1[i] == g_p[0, j] and Z2[i] == g_p[1, j] and Z3[i] == g_p[2, j]:
            ind = j
            break
    if ind != -1:
        for k in range(g.shape[0]):
            if g_p[k, ind] == 1:
                A_p[i, k] = 1  # assign to indicate positive part of contrast vector
            else:
                A_m[i, k] = 1  # assign to indicate negative part of contrast vector

We use a basis function $q_{sJ}(X_i, Z_i) = h_s(X_i) \Pi_{j \in J} Z_{ij}, s = 1,..., S, J \in [K]_{K'}$ Because we only consider the main effects non-negligible / non-zero, then $K' = 1$. The specific covariate basis function is $h_s(X) = X_s, s=1, ..., 5$, meaning the basis function used is just each of the five covariates themselves.

In [10]:
# we can just define them outright
h = np.copy(X)

The main effect of each treatment $z_k$, $\tau_k$ is a comparison between the average potential outcomes of receiving treatment $z_k$ and the average potential outcomes of note receiving it, averaged over all treatment combinations of other factors, expressed as:

$\tau_k = \frac{1}{2^{K-1}} g_k^T E[Y]$. 

Where $E[Y] = (E[Y(z)])_{z\in Z}$.

For the purposes of this simulation, we don't attempt to derive the interaction effects $\tau_{k, k'}$ which measure the interaction between two factors $z_k$ and $z_k'$.